In [ ]:

!pip uninstall torch -y

!pip install torch torchvision torchaudio


!pip install transformers datasets

!pip install scikit-learn

!pip install wandb


Found existing installation: torch 2.5.0+cu121
Uninstalling torch-2.5.0+cu121:
  Successfully uninstalled torch-2.5.0+cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.0 MB/s eta 0:00:00

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.optim import AdamW

os.environ['WANDB_DISABLED'] = 'true'

tokenizer = AutoTokenizer.from_pretrained("ltrctelugu/ltrc-albert")
model = AutoModelForSequenceClassification.from_pretrained("ltrctelugu/ltrc-albert", num_labels=3)

dataset = load_dataset("mounikaiiith/Telugu_Sentiment")

def encode_labels(example):
    label_mapping = {'neg': 0, 'neutral': 1, 'pos': 2}
    example['label'] = label_mapping[example['Sentiment']]
    return example

encoded_dataset = dataset.map(encode_labels)

def tokenize_function(example):
    return tokenizer(example['Sentence'], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = encoded_dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_dataset['train']
val_dataset = tokenized_dataset['validation']
test_dataset = tokenized_dataset['test']

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(AdamW(model.parameters(), lr=3e-5), None)  # Specify AdamW optimizer
)

trainer.train()

train_results = trainer.evaluate(train_dataset)
print("Training set evaluation:", train_results)

val_results = trainer.evaluate(val_dataset)
print("Validation set evaluation:", val_results)

test_results = trainer.evaluate(test_dataset)
print("Test set evaluation:", test_results)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Training set evaluation: {'eval_loss': 1.0017105340957642, 'eval_accuracy': 0.5371356559209723, 'eval_f1': 0.375392648983593, 'eval_precision': 0.2885147128616532, 'eval_recall': 0.5371356559209723, 'eval_runtime': 179.9705, 'eval_samples_per_second': 136.684, 'eval_steps_per_second': 4.273, 'epoch': 10.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Validation set evaluation: {'eval_loss': 1.0076806545257568, 'eval_accuracy': 0.5344729344729344, 'eval_f1': 0.37232500002644825, 'eval_precision': 0.2856613176841097, 'eval_recall': 0.5344729344729344, 'eval_runtime': 25.7001, 'eval_samples_per_second': 136.575, 'eval_steps_per_second': 4.28, 'epoch': 10.0}
Test set evaluation: {'eval_loss': 0.9956026673316956, 'eval_accuracy': 0.5430115171335135, 'eval_f1': 0.3821896395010851, 'eval_precision': 0.29486150773964, 'eval_recall': 0.5430115171335135, 'eval_runtime': 51.462, 'eval_samples_per_second': 136.664, 'eval_steps_per_second': 4.275, 'epoch': 10.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
